In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
A="A";B="B";C="C";D="D";E="E";F="F";G="G";H="H";I="I";J="J";K="K"
trans = [[A, B, C, D, E, F],[B, C, D, E, F, G],[A, D, E, H],[A, D, F, I, J],[B, D, E, K]]
transaction=dict()
s="T"
for i in range(0,5):
  b=s+str(i+1)
  transaction[b]=trans[i]


In [ ]:
min_support =3;min_confidence = 4;
transaction

{'T1': ['A', 'B', 'C', 'D', 'E', 'F'],
 'T2': ['B', 'C', 'D', 'E', 'F', 'G'],
 'T3': ['A', 'D', 'E', 'H'],
 'T4': ['A', 'D', 'F', 'I', 'J'],
 'T5': ['B', 'D', 'E', 'K']}

In [ ]:
vocab = set()
for i in trans:
  for j in i:
    vocab.add(j)
frequent = dict()
for i in vocab:
  frequent[i]=0

for i in trans:
  for j in i:
    frequent[j]+=1
frequent

{'D': 5,
 'G': 1,
 'H': 1,
 'E': 4,
 'J': 1,
 'B': 3,
 'F': 3,
 'K': 1,
 'C': 2,
 'I': 1,
 'A': 3}

In [ ]:
freq = dict()
for i in frequent:
  if frequent[i]>=min_support:
    freq[(i)]=frequent[i]
freq

{'D': 5, 'E': 4, 'B': 3, 'F': 3, 'A': 3}

In [ ]:
def candidate_generation(size,freq):
  new_candidates = dict()
  for items in freq:
    for item in freq:
      if len(set(items).union(set(item)))==size:
        candidate = tuple(sorted(set(items).union(set(item))))
        if candidate not in new_candidates:
          new_candidates[candidate]=0
  return new_candidates

In [ ]:
size=2
alpha = candidate_generation(size,freq)
while(alpha):
  for transactions_id in trans:
    for candidates in alpha:
      if set(candidates).issubset(set(transactions_id)):
        alpha[candidates]+=1

  freq.update({key:frequency for key,frequency in alpha.items() if frequency>=min_support})
  size+=1
  alpha = candidate_generation(size,freq)

In [ ]:
freq

{'D': 5,
 'E': 4,
 'B': 3,
 'F': 3,
 'A': 3,
 ('D', 'E'): 4,
 ('B', 'D'): 3,
 ('D', 'F'): 3,
 ('A', 'D'): 3,
 ('B', 'E'): 3,
 ('B', 'D', 'E'): 3}

In [ ]:
frozen_freq=dict()
for items,value in freq.items():
  frozen_freq[frozenset(items)]=value
frozen_freq

{frozenset({'D'}): 5,
 frozenset({'E'}): 4,
 frozenset({'B'}): 3,
 frozenset({'F'}): 3,
 frozenset({'A'}): 3,
 frozenset({'D', 'E'}): 4,
 frozenset({'B', 'D'}): 3,
 frozenset({'D', 'F'}): 3,
 frozenset({'A', 'D'}): 3,
 frozenset({'B', 'E'}): 3,
 frozenset({'B', 'D', 'E'}): 3}

In [ ]:
from itertools import combinations

def generate_association_rules(frequent_sets, min_confidence):
    association_rules = []
    for frequent_set in frequent_sets:
        for i in range(1, len(frequent_set)):
            for antecedent in combinations(frequent_set, i):
                antecedent = frozenset(antecedent)
                consequent = frequent_set - antecedent
                confidence = frequent_sets[antecedent.union(consequent)] / frequent_sets[antecedent]
                if confidence >= min_confidence:
                    association_rules.append((antecedent, consequent, confidence))
    return association_rules
min_confidence = 0.8
association_rules = generate_association_rules(frozen_freq, min_confidence)

In [ ]:
def generate_A_antecedent_rules(frequent_itemsets, min_confidence):
    A_antecedent_rules = []
    for antecedent, consequent, confidence in association_rules:
        if 'A' in antecedent:
            A_antecedent_rules.append((antecedent, consequent, confidence))
    return A_antecedent_rules


In [ ]:
for items in association_rules:
  print(set(items[0])," -> ",set(items[1]),"\t\t having confidence  = ", items[2])

{'E'}  ->  {'D'} 		 having confidence  =  1.0
{'D'}  ->  {'E'} 		 having confidence  =  0.8
{'B'}  ->  {'D'} 		 having confidence  =  1.0
{'F'}  ->  {'D'} 		 having confidence  =  1.0
{'A'}  ->  {'D'} 		 having confidence  =  1.0
{'B'}  ->  {'E'} 		 having confidence  =  1.0
{'B'}  ->  {'E', 'D'} 		 having confidence  =  1.0
{'E', 'B'}  ->  {'D'} 		 having confidence  =  1.0
{'D', 'B'}  ->  {'E'} 		 having confidence  =  1.0


In [ ]:
for items in generate_A_antecedent_rules(frozen_freq,min_confidence):
  print(set(items[0])," -> ",set(items[1]),"\t\t having confidence  = ", items[2])

{'A'}  ->  {'D'} 		 having confidence  =  1.0
